# Set up

In [41]:
import streamlit as st
import pandas as pd
import numpy as np
from datetime import date
import io
import random


import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [42]:
region_name = 'WDW'

In [43]:
parent_path = 'C:\\Users\\DONGZ027\\Desktop\\Local Crafts\\craft001_Soul'

mmmYear = 2023
ticket_price = 165

media_mapping_file = pd.read_csv(parent_path + '\data\media_label.csv')
media_mapping = media_mapping_file[media_mapping_file.region == region_name].set_index('media_code').to_dict()['media_label']

media_mapping_reverse = {v: k for k, v in media_mapping.items()}

spend_prefix = "M_P_"  
inc_prefix = "TIncT_P_" 
cpt_prefix = "CPT_P_"
mcpt_prefix = "MCPT_P_" 
minc_prefix = "nMTIncT_P_"

months_abbv = ['Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
months_full = ['October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September'] 

In [44]:
# Time reference
file_path = f"{parent_path}\data\DT_snowflake.csv"
time_ref = pd.read_csv(file_path)

time_ref = time_ref[[
    'FIS_WK_END_DT', 'FIS_YR_NB', 'FIS_QTR_NB', 'FIS_MO_NB', 'FIS_WK_NB', 
]].drop_duplicates() 
time_ref = time_ref[time_ref['FIS_YR_NB'] >= mmmYear]
time_ref['FIS_WK_END_DT'] = pd.to_datetime(time_ref['FIS_WK_END_DT']).dt.date
time_ref = time_ref.sort_values(by=['FIS_WK_END_DT']) 

counting_months = time_ref.groupby(['FIS_YR_NB', 'FIS_MO_NB']).size().reset_index(name='weeks_count') 
counting_months['lag_weeks'] = counting_months.groupby(['FIS_YR_NB'])['weeks_count'].cumsum().shift(fill_value=0) 
counting_months.loc[counting_months.FIS_MO_NB == 1, 'lag_weeks'] = 0

time_ref = time_ref.merge(counting_months, how = 'left', on = ['FIS_YR_NB', 'FIS_MO_NB']).drop_duplicates() 

counting_months2 = time_ref[['FIS_YR_NB', 'FIS_MO_NB']].drop_duplicates()
counting_months2 = counting_months2.reset_index(drop=True)
counting_months2['lag_months'] = counting_months2.index   

time_ref = time_ref.merge(counting_months2, how = 'left', on = ['FIS_YR_NB', 'FIS_MO_NB']).drop_duplicates() 
df_time = time_ref.reset_index()

In [60]:
# Media timing file
# **********************************************************************************
file_path = f"{parent_path}\\data\\{region_name}\\input_mediaTiming.csv"
df_curve  = pd.read_csv(file_path)
media_list = df_curve.columns


filler = []
for x in df_curve.columns:
    shard = np.zeros(104)
    values = df_curve[x].values
    print(x, values)
    shard[:len(values)] = values
    filler.append(shard)
df_curve = pd.DataFrame(np.array(filler).T, columns=df_curve.columns)
df_curve = df_curve.loc[~(df_curve == 0).all(axis=1)]
df_curve = df_curve.fillna(0) 

# Media parameters file
# **********************************************************************************
file_path = f"{parent_path}\\data\\{region_name}\\input_300pct.csv"
df_params  = pd.read_csv(file_path)
names = list(df_params.columns)
names2 = [s.replace("FABING", "ING") for s in names]
names2 = [s.replace("DIS_BAN", "BAN") for s in names2]
names2 = [s.replace("DIS_AFF", "AFF") for s in names2]
names2 = [s.replace("TlncT", "TIncT") for s in names2]
df_params.columns = names2

NTV [0.0029 0.0062 0.01   0.0144 0.0166 0.0192 0.0222 0.0256 0.0295 0.034
 0.0391 0.045  0.0517 0.0556 0.0566 0.0546 0.0494 0.0448 0.0405 0.0367
 0.0332 0.03   0.0272 0.0246 0.0223 0.0202 0.0183 0.0165 0.015  0.0135
 0.0123 0.0111 0.01   0.0091 0.0082 0.0075 0.0067 0.0061 0.0055 0.005
 0.0045 0.0041 0.0037 0.0034 0.003  0.0028 0.0025 0.0023 0.002  0.0019
 0.0017 0.0015 0.0014 0.0012 0.0011 0.001  0.0009 0.0008 0.0008 0.0007
 0.0006 0.0006 0.0005]
TTV [0.0029 0.0062 0.01   0.0144 0.0166 0.0192 0.0222 0.0256 0.0295 0.034
 0.0391 0.045  0.0517 0.0556 0.0566 0.0546 0.0494 0.0448 0.0405 0.0367
 0.0332 0.03   0.0272 0.0246 0.0223 0.0202 0.0183 0.0165 0.015  0.0135
 0.0123 0.0111 0.01   0.0091 0.0082 0.0075 0.0067 0.0061 0.0055 0.005
 0.0045 0.0041 0.0037 0.0034 0.003  0.0028 0.0025 0.0023 0.002  0.0019
 0.0017 0.0015 0.0014 0.0012 0.0011 0.001  0.0009 0.0008 0.0008 0.0007
 0.0006 0.0006 0.0005]
CTVP [0.0029 0.0062 0.01   0.0144 0.0166 0.0192 0.0222 0.0256 0.0295 0.034
 0.0391 0.045  0.0517 0

In [46]:
def processing_spend_df(file_path):
    data = pd.read_csv(file_path)

    data = data.T.iloc[1:, :]
    data.columns = media_mapping.keys()
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'FIS_MO_NB'}, inplace=True)
    data['FIS_MO_NB'] = np.arange(1, 13)
    return data


scenario1 = processing_spend_df(f"{parent_path}\\samples\WDW Demo\WDW_FY24.csv")
scenario2 = processing_spend_df(f"{parent_path}\\samples\WDW Demo\WDW_FY24_110.csv")
base_year = scenario1


# Mature Functions

In [47]:
def focal_check(X, spend_data, month, threshold):
    M_P_X_col = spend_prefix + X
    Inc_X_col = inc_prefix + X

    spend_array = spend_data[X].values
    S_total = spend_array.sum()
    benchmark_spend = 3 * S_total / np.count_nonzero(spend_array)

    # Handle the case when total spending is zero to avoid division by zero
    if S_total == 0:
        spend_percentages = np.zeros_like(spend_array)
    else:
        # Compute the percentage of total spend per month
        spend_percentages = spend_array / S_total
    multiplier = np.sum(spend_percentages >= (threshold / 100))

    m = spend_array[month - 1]
    M = min(m, benchmark_spend) * multiplier

    idx_closest = (np.abs(df_params[M_P_X_col] - M)).idxmin()
    M300 = df_params[M_P_X_col].iloc[idx_closest]
    Y300 = df_params[Inc_X_col].iloc[idx_closest]
    CPT300 = df_params[cpt_prefix + X].iloc[idx_closest]
    MA = df_params[minc_prefix + X].iloc[idx_closest]
    delta = df_params.at[idx_closest, 'Pct_Delta']
    MC =  delta * M

    y1 = Y300 / multiplier
    y2 = m / CPT300

    print(X, "at Month", month)
    print("multiplier = ", multiplier)
    print("monthly spend = ", m)
    print("benchmark spend = ", benchmark_spend)
    print(f"Annual index = {idx_closest}, Annual spend = {M}, Annual spend on table = {M300}")
    print(f"Closest M300: {M300}, Y300: {Y300}, CPT300: {CPT300}, MA: {MA}, MC: {MC}, delta: {delta}")
    print("y1 = ", y1, "y2 = ", y2)

    return y2

In [48]:
def compute_reward_X(X, spend_data, planning_year, threshold):
    """
    Compute the rewards over a 104-week period for a given media type and calculate
    marginal increments and costs, with an updated multiplier calculation based on a threshold.

    Args:
        X (str): The media type (e.g., "X1").
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'X1', 'X2', 'X3').
        planning_year (int): The fiscal year of the current spending plan.
        threshold (float): A value between 0 and 100. Months with spending percentage
            greater than threshold/100 will be counted towards the multiplier.

    Returns:
        tuple: A tuple containing:
            - reward_df (pd.DataFrame): DataFrame of shape (104, 13) with the reward curves.
            - minc_X (float): Total marginal rewards for media X over 12 months.
            - mc_X (float): Total marginal costs for media X over 12 months.
    """
    # Ensure global access to necessary dataframes
    global df_curve, df_params, df_time

    # ================================================================================================================
    # Step 1: Extract the spending array and compute the multiplier
    # ================================================================================================================
    spend_array = spend_data[X].values  # Spending for media X over 12 months
    # Compute total spending S_total
    S_total = spend_array.sum()
    benchmark_spend = 3 * S_total / np.count_nonzero(spend_array) # Compute the benchmark, or upper bound, for monthly spending

    # Handle the case when total spending is zero to avoid division by zero
    if S_total == 0:
        spend_percentages = np.zeros_like(spend_array)
    else:
        # Compute the percentage of total spend per month
        spend_percentages = spend_array / S_total

    # Compute the multiplier based on the threshold
    multiplier = np.sum(spend_percentages >= (threshold / 100))

    # If multiplier is zero, set it to 1 to avoid division by zero later
    if multiplier == 0:
        multiplier = 1

    # ================================================================================================================
    # Step 2: Compute the timing stuff
    # ================================================================================================================
    # Prepare a list to hold the 104-length arrays for each month
    monthly_arrays = []

    # Get fiscal years and months from spend_data
    fiscal_months = spend_data.FIS_MO_NB.values.flatten()

    # Filter df_time for the relevant fiscal year
    df_time_filtered = df_time[df_time['FIS_YR_NB'] == planning_year]

    # Group by fiscal month and count the number of weeks in each month
    weeks_in_month = df_time_filtered.groupby('FIS_MO_NB')['FIS_WK_NB'].nunique()
    weeks_in_month = weeks_in_month.reindex(range(1, 13), fill_value=0)

    # Compute cumulative weeks to determine the starting week for each month
    cumulative_weeks = weeks_in_month.cumsum()
    start_weeks = [0] + cumulative_weeks.values.tolist()[:-1]

    timeline = df_time[df_time.FIS_YR_NB.between(planning_year, planning_year + 2)].shape[0]
    timeline = np.zeros(timeline)

    # Initialize total marginal increments and costs
    minc_X = 0.0
    mc_X = 0.0

    # ================================================================================================================
    # Loop over each month to compute the rewards
    # ================================================================================================================
    for idx, S_mo in enumerate(spend_array):
        fiscal_month = fiscal_months[idx]

        if S_mo == 0:
            # If the spending is zero, append an array of zeros
            monthly_array = timeline.copy()  # Create a an array of all 0s
            monthly_arrays.append(monthly_array)
            continue

        # Step 2: Calculate annualized spending and find the closest match in df_params
        S_yr = min(S_mo, benchmark_spend) * multiplier
        M_P_X_col = spend_prefix + X
        TIncT_P_X_col = inc_prefix + X
        nMTIncT_P_X_col = minc_prefix + X  # Marginal rewards column
        Pct_Delta_col = 'Pct_Delta'   

        # Find the index of the closest spending value in df_params
        idx_closest = (np.abs(df_params[M_P_X_col] - S_yr)).idxmin()

        if idx_closest == 0:
            reward_mo = 0
        else:
            cpt_yr = df_params[cpt_prefix + X].iloc[idx_closest]
            reward_mo = S_mo / cpt_yr

        # Calculate minc_mo and mc_mo for marginal cost per reward
        minc_mo = df_params.at[idx_closest, nMTIncT_P_X_col]
        mc_mo = df_params.at[idx_closest, Pct_Delta_col] * S_yr
        # mc_mo = 0.001 * S_yr


        # Accumulate total marginal increments and costs
        minc_X += minc_mo
        mc_X += mc_mo

        # Step 3: Generate the reward curve for the month
        timing_curve = df_curve[X].values  # Timing curve of length 52
        monthly_reward_curve = reward_mo * timing_curve
        # monthly_arrays.append(monthly_reward_curve)

        # Step 4: Create a 3yr-length array with appropriate leading zeros
        start_week = start_weeks[fiscal_month - 1]
        end_week = start_week + len(monthly_reward_curve) 

        monthly_array = timeline.copy()  
        monthly_array[start_week:end_week] = monthly_reward_curve 
        monthly_arrays.append(monthly_array)
    
    
    # # #Aggregate the monthly arrays into a final reward curve
    # # monthly_arrays = [arr[:len(timeline)] for arr in monthly_arrays]  # Ensure each array is at most 104 elements
    # # # Pad any shorter arrays to exactly 104 elements
    # # monthly_arrays = [np.pad(arr, (0, len(timeline) - len(arr)), 'constant') if len(arr) < len(timeline) else arr for arr in monthly_arrays]
    reward_X = np.sum(monthly_arrays, axis=0)


    # Construct the output DataFrame
    columns = [f'Month_{i+1}' for i in range(12)] + ['aggregated']
    data = np.column_stack(monthly_arrays + [reward_X])
    reward_df = pd.DataFrame(data, columns=columns)

    return reward_df, minc_X, mc_X




In [49]:
def exmaine_media(media, spend_plan, year, ratio):
    time = df_time[df_time.FIS_YR_NB.between(year, year + 2)][['FIS_YR_NB', 'FIS_WK_NB', 'FIS_MO_NB']]
    time["Month"] = time['FIS_YR_NB'].astype(str) + '-' + time['FIS_MO_NB'].astype(str)


    time2 =  df_time[df_time.FIS_YR_NB.between(year, year + 2)][['FIS_YR_NB', 'FIS_MO_NB']] .drop_duplicates()
    time2 = time2.reset_index(drop=True)
    time2.reset_index(inplace=True)
    time = time2.merge(time, how='left', on=['FIS_YR_NB', 'FIS_MO_NB'])

    check = compute_reward_X(media, spend_plan, year, ratio)[0]
    items = check.columns
    check['Month'] = time['Month'].values
    check['Ind'] = time['index'].values
    check.sort_values(by=['Ind'], inplace=True)

    check = check.groupby(['Ind', 'Month'])[items].sum().reset_index()
    check.iloc[:, 1:] = np.round(check.iloc[:, 1:], 1)
    return check

In [50]:
def compute_plan_reward(spend_data, planning_year, lead_years, lag_years, threshold):
    """
    Compute the reward curves for all medias over an extended time frame, broken down by month and aggregated.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'X1', 'X2', 'X3').
        lead_years (int): Number of years to add as leading zeros.
        lag_years (int): Number of years to add as trailing zeros.2
        planning_year (int): The fiscal year of the current spending plan.
        threshold (float): Threshold value for reward calculations.

    Returns:
        list: A list of 13 numpy arrays:
            - First 12 arrays represent the monthly rewards summed across all media
            - Last array represents the aggregated rewards summed across all media
            Each array includes leading and trailing zeros.
    """
    # Ensure global access to necessary dataframes
    # global df_curve, df_params, df_time

    # Step 1: Get the list of media columns from spend_data
    medias = spend_data.columns.tolist()[1:]

    # Initialize lists to store the monthly and aggregated reward arrays
    monthly_rewards = [[] for _ in range(12)]  # One list for each month
    aggregated_rewards = []  # List for aggregated rewards

    # Loop over each media to compute its reward curves
    for media in medias:
        # Extract spending data for the media
        media_spend_data = spend_data[['FIS_MO_NB', media]]

        # Call compute_reward_X for the current media
        reward_df = compute_reward_X(media, media_spend_data, planning_year, threshold)[0]

        # Extract monthly columns and aggregated column
        for month in range(12):
            month_col = f'Month_{month+1}'
            monthly_rewards[month].append(reward_df[month_col].values)
        
        # Extract and store aggregated rewards
        aggregated_rewards.append(reward_df['aggregated'].values)

    # Step 2: Sum the reward arrays across all media for each month and aggregated
    summed_monthly_rewards = [np.sum(month_arrays, axis=0) for month_arrays in monthly_rewards]
    summed_aggregated_rewards = np.sum(aggregated_rewards, axis=0)

    # Step 3: Add leading and trailing zeros to each array
    leading_zeros = np.zeros(lead_years * 52)
    trailing_zeros = np.zeros(lag_years * 52)

    # Create final list of extended arrays (12 monthly + 1 aggregated)
    extended_reward_curves = []
    
    # Process monthly arrays
    for monthly_reward in summed_monthly_rewards:
        extended_monthly = np.concatenate([leading_zeros, monthly_reward, trailing_zeros])
        extended_reward_curves.append(extended_monthly)
    
    # Process aggregated array
    extended_aggregated = np.concatenate([leading_zeros, summed_aggregated_rewards, trailing_zeros])
    extended_reward_curves.append(extended_aggregated)

    return extended_reward_curves


In [51]:
def plan_forecast_craft(spend_data, planning_year, lead_years, lag_years, cutoff):
    """
    Generate monthly and quarterly forecast tables based on spending data and reward calculations.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'NTV', 'ING', 'STR').
        planning_year (int): The fiscal year of the current spending plan.
        lead_years (int): Number of years to add as leading zeros.
        lag_years (int): Number of years to add as trailing zeros.
        cutoff (float): Threshold value for reward calculations.

    Returns:
        tuple: A tuple containing:
            - craft_mo (pd.DataFrame): DataFrame with monthly forecast data.
            - craft_qtr (pd.DataFrame): DataFrame with quarterly forecast data.
    """
    df_time_scenario = df_time[df_time['FIS_YR_NB'].between(planning_year - lead_years, planning_year + lag_years)]
    weekly_table = df_time_scenario[['FIS_WK_END_DT', 'FIS_YR_NB', 'FIS_QTR_NB', 'FIS_MO_NB']]
    results = compute_plan_reward(spend_data, planning_year, lead_years, lag_years, cutoff)

    names = []
    for i in range(len(results)-1):
        serl = list(results[i])
        if len(serl) < weekly_table.shape[0]:
            serl = serl + [0] * (weekly_table.shape[0] - len(serl))
        if len(serl) > weekly_table.shape[0]:
            serl = serl[:weekly_table.shape[0]]
        col_name = str(planning_year) + ' ' + months_abbv[i]
        names.append(col_name)
        weekly_table[col_name] = serl

    # Monthly Table
    # ================================================================================================================
    monthly_table = weekly_table.groupby(['FIS_YR_NB', 'FIS_MO_NB'])[names].sum().reset_index()
    rewards = monthly_table.iloc[:, 2:].values.T
    monthly_table.FIS_MO_NB.replace(dict(zip(range(1, 13), months_abbv)), inplace=True) 
    monthly_table['timeline'] = monthly_table.FIS_YR_NB.astype(str) + " " + monthly_table.FIS_MO_NB.astype(str) 
    
    shard1 = pd.DataFrame({'Spending Month': names, "Spend": spend_data.iloc[:, 1:].sum(axis = 1).values})
    shard2 = pd.DataFrame(rewards)
    shard2.columns = monthly_table['timeline'].values
    craft_mo = pd.concat([shard1, shard2], axis=1) 


    # Monthly Table
    # ================================================================================================================
    quarter_table = weekly_table.groupby(['FIS_YR_NB', 'FIS_QTR_NB'])[names].sum().reset_index()
    rewards = quarter_table.iloc[:, 2:].values.T 
    rewards = rewards.reshape(4, 3, -1) # Turning monthly tracking into quarterly tracking
    rewards = rewards.sum(axis = 1)
    quarter_table.FIS_QTR_NB = quarter_table.FIS_QTR_NB.astype(str)
    quarter_table['timeline'] = quarter_table.FIS_YR_NB.astype(str) + " Q" + quarter_table.FIS_QTR_NB.astype(str)

    names = [str(planning_year) + " Q" + str(x) for x in range(1, 5)]
    shard1 = pd.DataFrame({'Spending Quarter': names, 
                           "Spend": spend_data.iloc[:, 1:].values.sum(axis = 1).reshape(4, 3).sum(axis = 1)})
    shard2 = pd.DataFrame(rewards)
    shard2.columns = quarter_table['timeline'].values
    craft_qtr = pd.concat([shard1, shard2], axis=1) 


    return craft_mo, craft_qtr
    

In [55]:
df_curve

,NTV,TTV,CTVP,CTVNP,DD,STRP,STRNP,YOT,CIN,DISP,DISNP,SEM,PAID_SOCIAL,DCI,DRD
0,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029,0.0029
1,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062,0.0062
2,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100
3,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144,0.0144
4,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166,0.0166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.0008,0.0008,NaN,0.0008,NaN,0.0008,0.0008,NaN,0.0008,NaN,0.0008,0.0008,0.0008,0.0008,NaN
59,0.0007,0.0007,NaN,0.0007,NaN,0.0007,0.0007,NaN,0.0007,NaN,0.0007,0.0007,0.0007,0.0007,NaN
60,0.0006,0.0006,NaN,0.0006,NaN,0.0006,0.0006,NaN,0.0006,NaN,0.0006,0.0006,0.0006,0.0006,NaN
61,0.0006,0.0006,NaN,0.0006,NaN,0.0006,0.0006,NaN,0.0006,NaN,0.0006,0.0006,0.0006,0.0006,NaN


In [62]:
plan_forecast_craft(scenario1, 2025, 1, 2, 0.46)[1]

,Spending Quarter,Spend,2024 Q1,2024 Q2,2024 Q3,2024 Q4,2025 Q1,2025 Q2,2025 Q3,2025 Q4,2026 Q1,2026 Q2,2026 Q3,2026 Q4,2027 Q1,2027 Q2,2027 Q3,2027 Q4
0,2025 Q1,47548717.12189,0.0,0.0,0.0,0.0,292544.964391,922225.208536,4.031424e+05,1.104602e+05,28583.806505,2533.422884,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,2025 Q2,61557399.86867,0.0,0.0,0.0,0.0,0.000000,324988.633542,1.010021e+06,4.177304e+05,114456.417770,29462.661382,2344.128825,0.000000,0.000000,0.0,0.0,0.0
2,2025 Q3,65661321.72848,0.0,0.0,0.0,0.0,0.000000,0.000000,3.290941e+05,1.029052e+06,426298.565741,116803.002575,30066.062707,2409.596320,0.000000,0.0,0.0,0.0
3,2025 Q4,45984315.2809,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,2.907802e+05,909915.709026,378538.433612,103715.206833,27338.336698,1487.090186,0.0,0.0,0.0


In [32]:
def build_plan_summary(spend_data, planning_year, threshold, unit_revenue):
    """
    Build a summary report of the spending plan with total spend, total reward,
    cost per reward, and marginal cost per reward for each media and aggregated.

    Args:
        spend_data (pd.DataFrame): DataFrame containing spending data with columns:
            - 'FIS_MO_NB': Fiscal month number.
            - Media spending columns (e.g., 'YOT', 'FAB').
        planning_year (int): The fiscal year of the current spending plan.

    Returns:
        pd.DataFrame: A DataFrame 'plan_summary' with columns:
            - 'Total Spend'
            - 'Total Reward'
            - 'Cost per Reward'
            - 'Marginal Cost per Reward'
        and rows:
            - 'Total' (aggregated over all medias)
            - One row per media (e.g., 'YOT', 'FAB')
    """
    # Ensure nonlocal access to necessary dataframes
    # nonlocal df_curve, df_params, df_time

    # Initialize dictionaries to store summary data
    total_spend_dict = {}
    total_reward_dict = {}
    minc_X_dict = {}
    mc_X_dict = {}

    # List of media columns (exclude 'FIS_MO_NB')
    medias = spend_data.columns.tolist()[1:]

    # Loop over each media to compute summary metrics
    for media in medias:
        # Extract spending data for the media
        media_spend_data = spend_data[['FIS_MO_NB', media]]


        # Total spend for media: sum of monthly spend
        total_spend = media_spend_data[media].sum()
        total_spend_dict[media] = total_spend


        # Call compute_reward_X for the current media
        reward_df, minc_X, mc_X = compute_reward_X(media, media_spend_data, planning_year, threshold)
        

        # Total reward for media: sum of the 'aggregated' column
        total_reward = reward_df['aggregated'].sum()
        total_reward_dict[media] = total_reward

        # Store minc_X and mc_X
        minc_X_dict[media] = minc_X
        mc_X_dict[media] = mc_X


    # Calculate aggregated totals
    total_spend_agg = sum(total_spend_dict.values())
    total_reward_agg = sum(total_reward_dict.values())
    minc_X_agg = sum(minc_X_dict.values())
    mc_X_agg = sum(mc_X_dict.values())


    # Build the summary DataFrame
    plan_summary = pd.DataFrame(columns=['Total Spend', 'Total Attendance', 'Cost per Attendance', 'Marginal Cost per Attendance'])

    # Add aggregated totals to the summary
    plan_summary.loc['Total'] = {
        'Total Spend': total_spend_agg,
        'Total Attendance': total_reward_agg,
        'Cost per Attendance': total_spend_agg / total_reward_agg if total_reward_agg != 0 else np.nan,
        'Marginal Cost per Attendance': mc_X_agg / minc_X_agg if minc_X_agg != 0 else np.nan
    }

    # Add individual media data to the summary
    for media in medias:
        total_spend = total_spend_dict[media]
        total_reward = total_reward_dict[media]
        minc_X = minc_X_dict[media]
        mc_X = mc_X_dict[media]

        plan_summary.loc[media] = {
            'Total Spend': total_spend,
            'Total Attendance': total_reward,
            'Cost per Attendance': total_spend / total_reward if total_reward != 0 else np.nan,
            'Marginal Cost per Attendance': mc_X / minc_X if minc_X != 0 else np.nan
        }

    # Reorder rows: aggregated totals first, then individual medias
    plan_summary = plan_summary.reset_index().rename(columns={'index': 'Media'})
    plan_summary = plan_summary[['Media', 'Total Spend', 'Total Attendance', 'Cost per Attendance', 'Marginal Cost per Attendance']]

    # Add ROAS and MROAS
    plan_summary['ROAS'] = unit_revenue * plan_summary['Total Attendance'] / plan_summary['Total Spend']
    plan_summary['MROAS'] = 1 + ((unit_revenue - plan_summary['Marginal Cost per Attendance']) / plan_summary['Marginal Cost per Attendance'])


    # Round up columns
    plan_summary['Total Spend'] = plan_summary['Total Spend'].astype(int)
    plan_summary['Total Attendance'] = plan_summary['Total Attendance'].astype(int)
    plan_summary['Cost per Attendance'] = plan_summary['Cost per Attendance'].round(1)
    plan_summary['Marginal Cost per Attendance'] = plan_summary['Marginal Cost per Attendance'].round(1)
    plan_summary['ROAS'] = plan_summary['ROAS'].round(1)
    plan_summary['MROAS'] = plan_summary['MROAS'].round(1)


    return plan_summary

In [33]:
build_plan_summary(scenario1, 2025, 0.46, ticket_price)

,Media,Total Spend,Total Attendance,Cost per Attendance,Marginal Cost per Attendance,ROAS,MROAS
0,Total,220751753,7313323,30.2,23745.4,5.5,0.0
1,NTV,34285760,813710,42.1,27474.2,3.9,0.0
2,TTV,8371501,227353,36.8,25565.3,4.5,0.0
3,CTVP,1888728,50169,37.6,18485.5,4.4,0.0
4,CTVNP,5288366,155580,34.0,24309.4,4.9,0.0
5,DD,14006112,369625,37.9,24217.7,4.4,0.0
6,STRP,12150034,418161,29.1,20481.3,5.7,0.0
7,STRNP,30495772,960916,31.7,25815.1,5.2,0.0
8,YOT,10992749,474626,23.2,18956.2,7.1,0.0
9,CIN,6576649,188332,34.9,25894.9,4.7,0.0


In [21]:
plan0 = scenario1.copy()
planning_months = [7, 8, 9]

int(plan0[plan0.FIS_MO_NB.isin(planning_months)].iloc[:, 1:].values.sum())


65661321

In [22]:
def optimization_summary(plan0, plan1, planning_year, planning_months, rewards, cutoff, unit_revenue):

    # Table 1 - Aggregate Summary
    # ****************************************************************************************************************
    contents = ['Total Spend', 'Total Reward', 'Total Reward in Planning Period', 'Cost per Reward', 'Marginal Cost per Reward', 'ROAS', "MROAS"]
    summary0 = build_plan_summary(plan0, planning_year, cutoff, unit_revenue)
    summary1 = build_plan_summary(plan1, planning_year, cutoff, unit_revenue)
    spend0 = int(plan0[plan0.FIS_MO_NB.isin(planning_months)].iloc[:, 1:].values.sum())
    spend1 = int(plan1[plan1.FIS_MO_NB.isin(planning_months)].iloc[:, 1:].values.sum())

    total_original = summary0.iloc[0, 1:].values.tolist()
    planning_original = rewards[0]
    collect_original = total_original[:2] + [planning_original] + total_original[2:]

    total_optimized = summary1.iloc[0, 1:].values.tolist()
    planning_optimized = rewards[1]
    collect_optimized = total_optimized[:2] + [planning_optimized] + total_optimized[2:]

    table1 = pd.DataFrame({
        'Contents': contents,
        'Original': collect_original,
        'Optimized': collect_optimized
    })

    array_original = table1.Original.values
    array_optimized = table1.Optimized.values

    
    array_original[0] = spend0
    array_original[3] = np.round(array_original[0] / array_original[2], 1) 
    array_original[5] = np.round(unit_revenue * array_original[2] / array_original[0], 1)
    table1['Original'] = array_original

    array_optimized[0] = spend1
    array_optimized[3] = np.round(array_optimized[0] / array_optimized[2], 1)
    array_optimized[5] = np.round(unit_revenue * array_optimized[2] / array_optimized[0], 1)
    table1['Optimized'] = array_optimized


    # table1.loc[table1.Contents == 'Cost per Reward', 'Original'] = np.round(table1.loc[table1.Contents == 'Total Spend', 'Original'] / table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Original'], 1)
    # table1.loc[table1.Contents == 'Cost per Reward', 'Optimized'] = np.round(table1.loc[table1.Contents == 'Total Spend', 'Optimized'] / table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Optimized'], 1)

    # table1.loc[table1.Contents == 'ROAS', 'Original'] =  np.round((unit_revenue * table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Original'].values[0]) / table1.loc[table1.Contents == 'Total Spend', 'Original'], 1)
    # table1.loc[table1.Contents == 'ROAS', 'Optimized'] =  np.round((unit_revenue * table1.loc[table1.Contents == 'Total Reward in Planning Period', 'Optimized'].values[0]) / table1.loc[table1.Contents == 'Total Spend', 'Optimized'], 1)


    table1['Change'] = table1['Optimized'] - table1['Original']
    table1['Change (%)'] = np.round((table1.Optimized / table1.Original - 1) * 100, 1)


    # Table 2 - Media Level Summary
    # ****************************************************************************************************************
    values = []
    for x in media_list:
        collect0 = np.round(summary0[summary0.Media == x].iloc[:, 1:].values, 1)[0]
        spend0 = collect0[0]
        reward0_total = collect0[1]
        reward0_plan = np.round(compute_reward_X(x, plan0, planning_year, cutoff)[0]['aggregated'].values[:52].sum(),1) 
        cpa0 = collect0[2]
        mcpa0 = collect0[3]
        values.append([x, 'original', spend0, reward0_total, reward0_plan, cpa0, mcpa0])

        collect1 = np.round(summary1[summary1.Media == x].iloc[:, 1:].values, 1)[0]
        spend1 = collect1[0]
        reward1_total = collect1[1]
        reward1_plan = np.round(compute_reward_X(x, plan1, planning_year, cutoff)[0]['aggregated'].values[:52].sum(),1)
        cpa1 = collect1[2]
        mcpa1 = collect1[3]
        values.append([x, 'optimized', spend1, reward1_total, reward1_plan, cpa1, mcpa1])

    table2 = pd.DataFrame(values, columns=['Media', 'Version', 'Total Spend', 'Total Reward', 'Total Reward in Planning Period', 'Cost per Reward', 'Marginal Cost per Reward'])
    table2.iloc[:, 2:] = np.round(table2.iloc[:, 2:], 1)
    
    craft = pd.DataFrame(table2['Media'])
    for x in table2.columns[2:]:
        df = table2[['Media', 'Version', x]]
        hierarchical_df = df.set_index(['Media', 'Version']).unstack(level=1)
        hierarchical_df.columns = [f'{col[0]} ({col[1]})' for col in hierarchical_df.columns]
        hierarchical_df = hierarchical_df.reset_index()
        hierarchical_df[x + ' Change(%)'] = 100 * (hierarchical_df[x + ' (optimized)'] / hierarchical_df[x + ' (original)'] - 1)
        hierarchical_df[x + ' Change(%)'] = np.round(hierarchical_df[x + ' Change(%)'], 1)
        craft = craft.merge(hierarchical_df, how = 'left', on = "Media").drop_duplicates()
    table2 = craft

    return [plan1, table1, table2]

In [ ]:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [25]:
optimization_summary(scenario1, scenario1, 2024, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [7313323, 7313323], 0.46, 165)[1]

,Contents,Original,Optimized,Change,Change (%)
0,Total Spend,220751753.0,220751753.0,0.0,0.0
1,Total Reward,7313323.0,7313323.0,0.0,0.0
2,Total Reward in Planning Period,7313323.0,7313323.0,0.0,0.0
3,Cost per Reward,30.2,30.2,0.0,0.0
4,Marginal Cost per Reward,23745.4,23745.4,0.0,0.0
5,ROAS,5.5,5.5,0.0,0.0
6,MROAS,0.0,0.0,0.0,NaN


# Scenrio Plots

In [53]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [54]:
l1 = [1, 2, 3]
l2 = [2, 1, 2]
[max(l1[i], l2[i]) for i in np.arange(len(l1))]

[2, 2, 3]

In [55]:
np.arange(3)

array([0, 1, 2])

You can find the intersection of two sets using the `intersection()` method or the `&` operator in Python. Here is an example:

```python
# Define two sets
set1 = {1, 2, 3, 4, 5}
set2 = {4, 5, 6, 7, 8}

# Find the intersection using the intersection() method
intersection_set = set1.intersection(set2)
print(intersection_set)

# Find the intersection using the & operator
intersection_set = set1 & set2
print(intersection_set)
```

Both methods will give you the intersection of the two sets.

In [71]:
def scenario_plots(scenarios, metrics, channels, colors, title, ylabel1, ylabel2, currency_symbol):
    # **********************************************************************************
    # Read numbers
    # **********************************************************************************
    board1 = scenarios[0]
    board2 = scenarios[1]

    metric1 = metrics[0]
    metric2 = metrics[1]

    # Drop medias with no spending
    # ******************************************************************************
    drops = []
    set1 = set(board1.loc[board1[metric1] == 0, 'Media'].values) 
    set2 = set(board2.loc[board2[metric1] == 0, 'Media'].values)
    drops.append(list(set1.intersection(set2)))
    board1 = board1[~board1['Media'].isin(drops[0])]
    board2 = board2[~board2['Media'].isin(drops[0])] 
    channels = [x for x in channels if x not in drops[0]]


    # Set up metrics
    # ********************************************************************************** 
    metric1_s1 = board1[metric1].values
    metric1_s2 = board2[metric1].values
    max_metric1 =  max(max(metric1_s1), max(metric1_s2))

    metric2_s1 = board1[metric2].values
    metric2_s2 = board2[metric2].values
    max_metric2 = max(max(metric2_s1), max(metric2_s2))

    metric1_compare = [(s2 - s1) / s1 * 100 for s1, s2 in zip(metric1_s1, metric1_s2)]
    metric2_compare = [(s2 - s1) / s1 * 100 for s1, s2 in zip(metric2_s1, metric2_s2)]

    color1_s1 = colors[0]
    color1_s2 = colors[1]
    color2_s1 = colors[2]
    color2_s2 = colors[3]

    default_textpos = 0.5 * max_metric1

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # **********************************************************************************
    # The bar plots
    # **********************************************************************************
    # Scenario 1
    # ....................................................
    fig.add_trace(
        go.Bar(
            x= channels,
            y= metric1_s1,
            name= "1",
            marker_color= color1_s1
        ),
        secondary_y=False,
    )

    # Scenario 2
    # ....................................................
    fig.add_trace(
        go.Bar(
            x= channels,
            y= metric1_s2,
            name= "2",
            marker_color= color1_s2
        ),
        secondary_y=False,
    )




    # **********************************************************************************
    # Line plots
    # **********************************************************************************
    # Scenario 1
    # ....................................................
    fig.add_trace(
        go.Scatter(
            x=channels,
            y= metric2_s1,
            name="MROAS 1",
            line=dict(
                color= color2_s1, 
                width=3, dash='dash',
                shape='spline',  # This creates a smooth curve
                smoothing=1.3    # Adjust smoothing intensity (0.5-1.5 range works well)
                ),
            marker=dict(
                size=10,         # Larger marker size
                color=color2_s1,
                line=dict(
                    width=1,
                    color='white'
                )
            )

        ),
        secondary_y=True,
    )

    # Scenario 2
    # ....................................................
    fig.add_trace(
        go.Scatter(
            x=channels,
            y= metric2_s2,
            name="MROAS 2",
            line=dict(color= color2_s2, 
                      width=3, dash='dash',
                      shape='spline',  # This creates a smooth curve
                      smoothing=1.3    # Adjust smoothing intensity (0.5-1.5 range works well)
                ),
            marker=dict(
                size=10,         # Larger marker size
                color=color2_s2,
                line=dict(
                    width=1,
                    color='white'
                )
            )
        ),
        secondary_y=True,
    )




    # **********************************************************************************
    # Costmetics
    # **********************************************************************************

    # Annotations for metric 1 change
    for i, channel in enumerate(channels):
        change = metric1_compare[i]
        
        # Determine color based on change
        color = "green" if change >= 0 else "red"
        
        # Format text with plus/minus sign and percentage
        if change >= 0:
            text = f"+{change:.1f}%"  # Add plus sign for positive changes
        else:
            text = f"{change:.1f}%"   # Negative sign is automatically included
        
        # Improved positioning logic
        current_value = max(metric1_s1[i], metric1_s2[i])
        
        # If the value is very small (less than 5% of the maximum), use a fixed position
        if current_value < 0.03 * max_metric1:
            ypos = 0.12 * max_metric1  # Position at 15% of max height for very small values
        # If it's the maximum value, add a bit more space
        elif current_value >= 0.95 * max_metric1:
            ypos = 1.05 * max_metric1  # Position at 110% of max for the largest values
        # For medium values, position proportionally
        else:
            ypos = current_value + (0.125 * max_metric1)  # Position above the bar with consistent spacing
        
        # Add the annotation without arrows
        fig.add_annotation(
            x=channel,
            y=ypos, 
            text=text,
            showarrow=False,  # No arrow
            font=dict(
                color=color, 
                size=14,      # Slightly larger font for better visibility
                weight='bold' # Make it bold for emphasis
            ),
            align='center',
            bgcolor='rgba(255,255,255,0.7)',  # Semi-transparent white background
            bordercolor=color,
            borderwidth=1,
            borderpad=3
        )




    fig.update_layout(
        # Wider plot for spacing
        width=1300,
        height=700,
        # Extra large left margin
        margin=dict(t=80, r=50, b=100, l=150),
        # Title styling
        title=dict(
            text= title,
            font=dict(
                size=28,
                color= color1_s2,
                weight='bold'
            ),
            x=0.5
        ),
        # Other layout
        barmode='group',
        legend=dict(
            orientation="h",
            yanchor="bottom", 
            y=-0.15,
            xanchor="center",
            x=0.5
        ),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )

    # Set x-axis properties
    fig.update_xaxes(
        title_text="",
        showgrid=False,
        showline=True,
        linewidth=2,
        linecolor='lightgray'
    )

    # Set y-axes properties
    fig.update_yaxes(
        title_text= ylabel1,
        title_font=dict(size=16),
        range=[0, 1.2 * max(max(metric1_s2), max(metric1_s1))],
        showgrid=True,
        gridcolor='lightgray',
        secondary_y=False,
        tickformat=','
    )

    fig.update_yaxes(
        title_text= ylabel2,
        title_font=dict(size=16),
        range=[0, 1.2 * max(max(metric2_s2), max(metric2_s1))],
        showgrid=False,
        secondary_y=True,
        tickprefix= currency_symbol,
        ticksuffix='.0'
    )

    # Show the figure
    fig.show()

In [73]:
build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :]

,Media,Total Spend,Total Attendance,Cost per Attendance,Marginal Cost per Attendance,ROAS,MROAS
1,NTV,5678942,331380,17.1,62.7,6.1,1.7
2,STR,2205421,149220,14.8,34.3,7.1,3.1
3,BAN,410196,52691,7.8,16.3,13.5,6.4
4,RAD,265052,16515,16.0,29.7,6.5,3.5
5,AFF,77974,12020,6.5,11.0,16.2,9.6
6,CEM,0,0,NaN,NaN,NaN,NaN
7,SEM,1075893,118347,9.1,19.7,11.5,5.3
8,Total_Social,3902440,346163,11.3,26.9,9.3,3.9


In [72]:
scenario_plots(
    scenarios = [build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :], 
                 build_plan_summary(scenario1, 2024, 0.9, ticket_price).iloc[1:, :]],

    metrics = ['Total Spend', 'MROAS'],

    channels = media_list,

    colors = ['rgb(174, 139, 113)', 
            'rgb(140, 63, 12)',
            'rgb(174, 139, 113)',
            'rgb(140, 63, 12)'
            
            ],

    title = "Media budget & MROAS variation per touchpoint", 

    ylabel1 = "", ylabel2= "", currency_symbol = "£" 
)[0]

TypeError: 'NoneType' object is not subscriptable

In [25]:
scenario_plots(
    scenarios = [build_plan_summary(scenario1, 2024, 0.9, ticket_price).iloc[1:, :], 
                 build_plan_summary(scenario2, 2024, 0.9, ticket_price).iloc[1:, :]],

    metrics = ['Total Attendance', 'Cost per Attendance'],

    channels = media_list,

    colors = ['rgb(188, 214, 150)', 
            'rgb(36, 84, 40)',
            'rgb(188, 214, 150)', 
            'rgb(36, 84, 40)'
            ],

    title = "Incremental attendance & CPA evolution", 

    ylabel1 = "", ylabel2= "", currency_symbol = "£" 
)

NameError: name 'scenario_plots' is not defined

# Labeled plot

In [4]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add the gauge (just the arc/dial part)
fig.add_trace(go.Indicator(
    mode="gauge",
    value=11,
    gauge={
        'axis': {
            'range': [0, 100],
            'visible': True,
            'showticklabels': False,
            'tickwidth': 1,
            'tickcolor': "white"
        },
        'bar': {'color': "white"},
        'bgcolor': "rgba(0,0,0,0)",
        'borderwidth': 0,
        'bordercolor': "rgba(0,0,0,0)",
        'steps': [],
        'threshold': {
            'line': {'color': "white", 'width': 4},
            'thickness': 0.75,
            'value': 11
        }
    },
    domain={'x': [0, 1], 'y': [0, 0.4]}  # Position the gauge at the bottom
))

# Add the text elements with proper spacing
fig.add_annotation(
    text="<b>CPA variation</b>",
    x=0.5, y=0.8,  # Position at the top
    showarrow=False,
    font=dict(size=16, color="white")
)

fig.add_annotation(
    text="€1.4",
    x=0.5, y=0.65,  # Position below the title with space
    showarrow=False,
    font=dict(size=24, color="white")
)

fig.add_annotation(
    text="<b style='color:red'>11%</b>",
    x=0.5, y=0.5,  # Position below the value with space
    showarrow=False,
    font=dict(size=28, color="red")
)

# Update layout with dark background
fig.update_layout(
    paper_bgcolor="#333333",  # Dark gray background
    plot_bgcolor="rgba(0,0,0,0)",
    height=400,
    width=400,
    margin=dict(t=120, b=80, l=40, r=40),  # Add more margin to ensure everything fits
    showlegend=False
)

# Show the figure
fig.show()

# To save:
# fig.write_image("gauge_chart.png")
# fig.write_html("gauge_chart.html")

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with customized gauge
fig = go.Figure()

# Add the gauge
fig.add_trace(go.Indicator(
    mode="gauge",
    value=11,  # The percentage value (11%)
    gauge={
        'axis': {
            'range': [None, 100], 
            'visible': True,
            'showticklabels': False,
            'tickwidth': 1, 
            'tickcolor': "white"
        },
        'bar': {'color': "white"},
        'bgcolor': "rgba(0,0,0,0)",
        'borderwidth': 0,
        'bordercolor': "rgba(0,0,0,0)",
        'steps': [],
        'threshold': {
            'line': {'color': "white", 'width': 4},
            'thickness': 0.75,
            'value': 11
        }
    },
    domain={'x': [0, 1], 'y': [0, 0.6]}
))

# Add the textual components separately for more control
fig.add_annotation(
    text="<b>CPA variation</b>",
    x=0.5, y=0.7,
    showarrow=False,
    font=dict(size=16, color="white")
)

fig.add_annotation(
    text="€1.4",
    x=0.5, y=0.6,
    showarrow=False,
    font=dict(size=24, color="white")
)

fig.add_annotation(
    text="11%",
    x=0.5, y=0.4,
    showarrow=False,
    font=dict(size=24, color="red")
)

# Update layout
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    margin=dict(t=100, b=0, l=0, r=0),
    height=300,
    width=300,
    showlegend=False
)

# Show the figure
fig.show()

# To save as an image or HTML file:
# fig.write_image("gauge_chart.png")
# fig.write_html("gauge_chart.html")